In [1]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import csv
import pandas as pd
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.84.116.10:8470


W0823 01:05:28.809107 140492258375552 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 17264453172763702357),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3352902676190696243),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12906206933941431307),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4312019761890040872),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 12003719367335478604),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 13349362904099686832),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 9513597623860807959),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 4587023516144653857),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 13771961239067624360),
 _DeviceAttributes(/job:tpu_w

In [2]:
import sys
from os import listdir
!rm -rf bert_finetune
!test -d bert_finetune || git clone https://github.com/junhahyung/bert_finetune bert_finetune
if not 'bert_finetune' in sys.path:
  sys.path += ['bert_finetune']
print(listdir("bert_finetune"))
# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_finetune'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 74 (delta 29), reused 39 (delta 11), pack-reused 0
Unpacking objects: 100% (74/74), done.


W0823 01:05:34.308135 140492258375552 deprecation_wrapper.py:119] From bert_finetune/optimization.py:88: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



['run_pretraining.py', 'optimization_test.py', 'modeling_test.py', 'multilingual.md', 'LICENSE', 'extract_features.py', 'run_classifier_with_tfhub.py', 'README.md', '__init__.py', 'predicting_movie_reviews_with_bert_on_tf_hub.ipynb', 'run_squad.py', 'tokenization_test.py', '__init__.pyc', 'modeling.py', 'create_pretraining_data.py', 'run_classifier.py', '.gitignore', 'modeling.pyc', 'requirements.txt', 'sample_text.txt', 'tokenization.py', 'CONTRIBUTING.md', 'optimization.py', '.git']


In [3]:
TASK = 'korean_sa'
assert TASK in ('MRPC', 'CoLA', 'korean_sa'), 'Undefined task'

BUCKET = 'mbertfinetune' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
TASK_DATA_DIR = 'gs://{}/data/{}'.format(BUCKET, TASK)
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
print(tf.gfile.ListDirectory(TASK_DATA_DIR))
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

BERT_MODEL = 'multi_cased_L-12_H-768_A-12'
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

"""
tdir = os.path.join(TASK_DATA_DIR, "korean_train.csv")
with tf.gfile.Open(tdir, "r") as f:
  reader = pd.read_csv(tdir) 
  lines = []
  for line in reader:
    lines.append(line)
print(lines[18824])
"""

***** Task data directory: gs://mbertfinetune/data/korean_sa *****
['korean_dev.csv', 'korean_train.csv']
***** Model output directory: gs://mbertfinetune/bert-tfhub/models/korean_sa *****


'\ntdir = os.path.join(TASK_DATA_DIR, "korean_train.csv")\nwith tf.gfile.Open(tdir, "r") as f:\n  reader = pd.read_csv(tdir) \n  lines = []\n  for line in reader:\n    lines.append(line)\nprint(lines[18824])\n'

In [4]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
tokenizer.tokenize("한국어는 잘 안되는 것 같아요.")

W0823 01:05:55.280949 140492258375552 deprecation_wrapper.py:119] From bert_finetune/run_classifier_with_tfhub.py:151: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0823 01:05:55.926263 140492258375552 deprecation_wrapper.py:119] From bert_finetune/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



['한국', '##어는', '잘', '안', '##되는', '것', '같', '##아', '##요', '.']

In [5]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500
layer_wise_lr = (True, 0.3)
# layer_wise_lr = None 

processors = {
  "cola": run_classifier.ColaProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
  "korean_sa": run_classifier.KsaProcessor,
}
processor = processors[TASK.lower()]()
label_list = processor.get_labels()

processor.get_train_examples(TASK_DATA_DIR)
# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000 

def get_run_config(output_dir):  
  return tf.contrib.tpu.RunConfig(    
      cluster=tpu_cluster_resolver,    
      model_dir=output_dir,    
      save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,    
      tpu_config=tf.contrib.tpu.TPUConfig(        
          iterations_per_loop=ITERATIONS_PER_LOOP,        
          num_shards=NUM_TPU_CORES,        
          per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

W0823 01:05:56.588287 140492258375552 deprecation_wrapper.py:119] From bert_finetune/run_classifier.py:382: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



In [6]:
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR 

model_fn = run_classifier_with_tfhub.model_fn_builder(  
    num_labels=len(label_list),  
    learning_rate=LEARNING_RATE,  
    num_train_steps=num_train_steps,  
    num_warmup_steps=num_warmup_steps,  
    use_tpu=True,  
    bert_hub_module_handle=BERT_MODEL_HUB,
    layer_wise_lr=layer_wise_lr
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)

W0823 01:05:58.425448 140492258375552 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fc69765b7b8>) includes params argument, but params are not passed to Estimator.


In [0]:
# Train the model
def model_train(estimator):
  print('Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))

In [8]:
model_train(estimator_from_tfhub)

W0823 01:05:58.469501 140492258375552 deprecation_wrapper.py:119] From bert_finetune/run_classifier.py:837: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



Please wait...
***** Started training at 2019-08-23 01:06:00.554629 *****
  Num examples = 8961
  Batch size = 32


W0823 01:06:00.956728 140492258375552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
E0823 01:06:57.660384 140492258375552 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0823 01:06:57.663240 140492258375552 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0823 01:06:57.665940 140492258375552 tpu.py:376] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in t

***** Finished training at 2019-08-23 01:11:08.393651 *****


In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))

In [10]:
model_eval(estimator_from_tfhub)

***** Started evaluation at 2019-08-23 01:11:09.574261 *****
  Num examples = 2194
  Batch size = 8


E0823 01:11:21.593542 140492258375552 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0823 01:11:21.595629 140492258375552 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0823 01:11:21.598205 140492258375552 tpu.py:376] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0823 01:11:21.602598 140492258375552 tpu.py:376] Operation of type Placeholder (module_apply_tokens/mlm_positions) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0823 01:11:21.611754 140492258375552 tpu.py:376] Operation of type Placeholder (module_apply_tokens/bert/embeddings/word_embeddings) is not supported on the TPU. Execution will fail if this op is used in the grap

***** Finished evaluation at 2019-08-23 01:12:27.138484 *****
***** Eval results *****
  eval_accuracy = 0.58804744
  eval_loss = 1.1636988
  global_step = 840
  loss = 1.1987576
